# An algorithm to Learn Polytree Networks with Hidden nodes

## Import Necessary Packages

In [ ]:
# Standard library imports
from collections import defaultdict
from itertools import permutations
import copy

# Third-party imports
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pygraphviz
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import rpy2py
from scipy import stats
from scipy.stats import t
import statsmodels.api as sm


In [ ]:
'''
# Enable the conversion between Pandas DataFrames and R data.frames
pandas2ri.activate()
# Load and execute the R script
result = ro.r.source("/Users/takeshine/Desktop/node_number/25/tree_data.R")
returned_list = result[0]
# our synthetc data and edge list information for original polytree
x_data = rpy2py(returned_list[0])  
original_edge_data = rpy2py(returned_list[1])  
'''

In [ ]:
#x_data_path = "/Users/takeshine/Desktop/new_layer/data_set/x_data.csv"
#original_edge_data_path = "/Users/takeshine/Desktop/new_layer/data_set/original_edge_data.csv" 
# sample_size = sample_size
x_data = pd.read_csv(x_data_path)
df_data = x_data.sample(n=sample_size)

In [ ]:
# df_data = pd.DataFrame(x_data)
df_data.columns = [f'V{i+1}' for i in range(len(df_data.columns))]

In [ ]:
# Determine the number of columns
n = df_data.shape[1]
# Create a mapping of old column names to new column names
new_columns = {f"V{i+1}": f"y{i+1}" for i in range(n)}

# Rename the columns
df_data = df_data.rename(columns=new_columns)

In [ ]:
number_sample = df_data.shape[0]
alpha = 0.1  # Significance level
# Degrees of freedom
df = number_sample - 2
# Get critical t-value (two-tailed test)
critical_t_value = t.ppf(1 - alpha/2, df)

# Convert t-value to r (correlation coefficient)
threshold_corr = critical_t_value / (critical_t_value**2 + df)**0.5
threshold_corr

In [ ]:
# Calculate the original correlation matrix
original_correlation_matrix = df_data.corr()

# Use the `where` function to replace values, we use 0.05 as our cut
new_correlation_matrix  = df_data.corr().applymap(lambda x: 0 if abs(x) < threshold_corr else 1)

## Task 1: Pairwise-Finite Distance Algorithm (PFDA)

In [ ]:
def pairwise_finite_distance_algorithm(new_correlation_matrix):

    new_correlation_matrix = new_correlation_matrix.to_numpy()
    n = new_correlation_matrix.shape[0] # Get the number of nodes

    nodes = [f"y{i + 1}" for i in range(n)]
    sets = {} 

    # Step 1 - 3: for each node, if it's independent from all other nodes, add it to a set
    for i, node in enumerate(nodes):
        if all(new_correlation_matrix[i, np.arange(n) != i]== 0):
            sets[f'S_{i+1},0'] = {node}

    # Step 4 - 9 For each pair of nodes, if they are dependent, add them to a set
    for i, node in enumerate(nodes):
        for j, other in enumerate(nodes):
            if i < j and new_correlation_matrix[i,j] == 1:
                sets[f'S_{i+1},{j+1}'] = {node, other}
                while True:  # Keep adding nodes to the set as long as there exist nodes that are dependent on all nodes in the set
                    old_size = len(sets[f'S_{i+1},{j+1}'])

                    # For each other node, if it's dependent on all nodes in the set, add it to the set
                    for k, another in enumerate(nodes):
                        if k != i and k != j and all(new_correlation_matrix[k,nodes.index(n)] == 1 for n in sets[f'S_{i+1},{j+1}']):
                            sets[f'S_{i+1},{j+1}'].add(another)
                    if len(sets[f'S_{i+1},{j+1}']) == old_size:
                        break
                    
    # Step 10: Create a new dictionary to store the unique sets
    unique_sets = {}
    for key, value in sets.items():
        # Convert the set to a frozenset
        frozenset_value = frozenset(value)
        # Only add the set to the dictionary if it's not already in the dictionary
        if frozenset_value not in unique_sets.values():
            unique_sets[key] = frozenset_value

    # Convert the frozensets back to sets
    unique_sets = {key: set(value) for key, value in unique_sets.items()}

    return unique_sets

In [ ]:
unique_sets =  pairwise_finite_distance_algorithm(new_correlation_matrix)

In [ ]:
# Rename the output sets of task1 
task1_sets =  {}
# Interate through the unique sets, renaming them and optionally reordering the elements, preparing for task2 
for i, (key, value) in enumerate(unique_sets.items()):
    new_name = f'S_{i + 1}'
    sorted_value = sorted(value, key=lambda x: int(x[1:]))
    task1_sets[new_name] = sorted_value
    

## Task 2: Reconstruction Algorithm for Latent Rooted Trees

### Prepare inputs for Task 2

- Use partial correlations to represent conditonal independence statements
- Use linear regression to calcluate partial correlation:https://en.wikipedia.org/wiki/Partial_correlation
- we use t test to determine the cut out value for partial correlations

In [ ]:
# Create a global variable-to-index mapping
all_variables = [f'y{i}' for i in range(1, n+1)] # recall n is number of columns for df_data
var_to_idx = {var: idx for idx, var in enumerate(all_variables)}
global_partial_correlations = np.full((n,n,n),np.nan)

### Mathematical Explanation of Partial Correlation Calculation

This code calculates the partial correlations between pairs of variables while controlling for a third variable in a dataset.

1. **Partial Correlation**: 
   - Measures the correlation between two variables, controlling for one or more additional variables.

2. **Linear Regression for Residuals**: 
   - Linear regression models are fitted for each variable against the control variable.
   - Residuals (observed - predicted values) are then used for further calculation.

3. **Correlation of Residuals**: 
   - The partial correlation is calculated as the correlation between the residuals of two variables.

4. **Special Cases Handling**: 
   - If only one variable is present, partial correlation is 0.
   - With two variables and no control, it's the Pearson correlation.

5. **Storing Results**: 
   - Partial correlations are stored in a 3D array, indexed by the involved variables.

6. **Handling NaNs and Diagonal Values**: 
   - NaNs in the array are set to 0.
   - Diagonal values in the first two dimensions are set to 1 (correlation of a variable with itself).

In [ ]:
def calculate_partial_correlations_for_triplets(data,variables):

    '''
    data as dataframe 
    varibles should be columns of those samples we generated for each varible 
    '''

    # Iterate over all combinations of var1 and var2
    for var1, var2 in permutations(variables, 2):
        if len(variables) == 1:
            partial_correlation = 0 
        elif len(variables) == 2: 
            partial_correlation = data[var1].corr(data[var2])
        else:
            # For each combination of var1 and var2, iterate over the remaining variables as control_var
            for control_var in [var for var in variables if var != var1 and var != var2]:
                
                # Fit the linear regression model for var1 controlled by control_var
                model_var1 = sm.OLS(data[var1], sm.add_constant(data[control_var])).fit()
                residuals_var1 = model_var1.resid

                # Fit the linear regression model for var2 controlled by control_var
                model_var2 = sm.OLS(data[var2], sm.add_constant(data[control_var])).fit()
                residuals_var2 = model_var2.resid

                if np.std(residuals_var1) != 0 and np.std(residuals_var2) != 0:
                # Compute the correlation between the residuals
                    partial_correlation = residuals_var1.corr(residuals_var2)
                else:
                    partial_correlation = 0  # Set a default value
                # Store the result in the 3D array

                i, j, k = var_to_idx[var1], var_to_idx[var2], var_to_idx[control_var]
                global_partial_correlations[i, j, k] = partial_correlation

    # let us set nans to 0 first 
    global_partial_correlations[np.isnan(global_partial_correlations)] = 0

    # diagoal on first 2D should all be one
    for i in range(global_partial_correlations.shape[0]):
        for k in range(global_partial_correlations.shape[2]):
            global_partial_correlations[i,i,k] = 1

In [ ]:
# Calculate the partial correlations for each set
for set_name, variables in task1_sets.items():
    calculate_partial_correlations_for_triplets(df_data, variables)

### find the cut for partial correlations

In [ ]:
check_list = []
for i in range(global_partial_correlations.shape[0]):
    for j in range(global_partial_correlations.shape[1]):
        for k in range(global_partial_correlations.shape[2]):
            if (global_partial_correlations[i,j,k] != 1 and 
            global_partial_correlations[i,j,k] != 0 and 
            abs(global_partial_correlations[i,j,k])) < 0.3:
                check_list.append(abs(global_partial_correlations[i,j,k]))
print(len(check_list))
check_list.sort()

In [ ]:
def partial_corr_significance(r, n, k):
    # Compute the t-statistic
    t_stat = r * np.sqrt((n - k - 2) / (1 - r**2))
    
    # Compute the one-tailed p-value (for positive correlation)
    degree_of_freedom = n - k - 2
    p_value = 1 - stats.t.cdf(abs(t_stat), degree_of_freedom)
    
    return t_stat, p_value

In [ ]:
sample_size = df_data.shape[0] # Sample size
k = 1    # Number of control variables
p_value_paramater =0.05
for partial_corr in check_list:
    t_stat, p_value = partial_corr_significance(partial_corr, sample_size, k)
    if p_value < p_value_paramater: 
        #print(f"t-statistic: {t_stat:.4f}")
        
        # print(f"p-value: {p_value:.4f}")
        # print("Choose cut near " + str(partial_corr) )
        threshold = partial_corr
        break

In [ ]:
# We need to set entries whose absolute value is smaller our threshold to zero
global_partial_correlations[np.abs(global_partial_correlations) < threshold] = 0 

###  Task 2: determine the collapased quasi-skeleton of each rooted subtree

In [ ]:
def step3_find_terminal_node(V_temp, global_partial_correlations, variable_to_index):
    """
    Determine a visible terminal node in V_temp.

    Parameters:
    - V_temp: Current set of nodes.
    - global_partial_correlations: 3D numpy array of partial correlations.
    - variable_to_index: Dictionary mapping node names to indices.

    Returns:
    - terminal_node: The node with the least number of violating pairs.
    """
    violations_count = {}  # This dictionary will store count of violations for each node_k
    
    for node_k in V_temp:
        idx_k = variable_to_index[node_k]
        excluded_nodes = set(V_temp) - {node_k}
        count = 0
        for (node_i, node_j) in permutations(excluded_nodes, 2):
            if node_i != node_k and node_j != node_k:
                idx_i, idx_j = variable_to_index[node_i], variable_to_index[node_j]
                if global_partial_correlations[idx_i, idx_k, idx_j] != 0:
                    count += 1  # Count this as a violation for node_k
        violations_count[node_k] = count
    
    # Return the node with the least violations
    terminal_node = min(violations_count, key=violations_count.get)
    return terminal_node


In [ ]:
def step4_find_linked_node(terminal_node, V_temp, global_partial_correlations, var_to_idx):
    linked_node = None

    if terminal_node:
        idx_k = var_to_idx[terminal_node]
        found_node_l = False
        excluded_nodes_1 = set(V_temp) - {terminal_node}
        for node_l in excluded_nodes_1:
            idx_l = var_to_idx[node_l]
            is_linked = True
            for node_j in excluded_nodes_1:
                idx_j = var_to_idx[node_j]
                if idx_j != idx_l and global_partial_correlations[idx_k, idx_l, idx_j] == 0:
                    is_linked = False
                    break
            if is_linked:
                linked_node = node_l
                break

    # Return the updated V_temp and the linked_node
    return linked_node


In [ ]:
def compute_set_K(V_temp, terminal_node, global_partial_correlations, var_to_idx):
    K = []
    while True:
        K_updated = False
        for node_i, node_j in permutations(V_temp, 2):
            if node_i != node_j and node_i not in K:
                idx_i, idx_j = var_to_idx[node_i], var_to_idx[node_j]
                idx_k = var_to_idx[terminal_node]
                if global_partial_correlations[idx_j, idx_i, idx_k] != 0 and node_j not in K:
                    K.append(node_j)
                    K_updated = True
                    break
        if not K_updated:
            break
    return K

In [ ]:
def latent_tree_structure_prepare(set_of_nodes, global_partial_correlations, var_to_idx):
    edges = []
    latent_nodes = []
    V_temp = list(set_of_nodes)  # Convert to list for indexing.
    candidates = {}
    K = None # Initialize K as none 

    while True:
        # Step 2a: Check if V_temp has only two nodes
        if len(V_temp) == 2:
            edges.append((V_temp[0], V_temp[1]))
            break
        
        # Step 2b: If more than 2 nodes, proceed to find the terminal node
        terminal_node = step3_find_terminal_node(V_temp, global_partial_correlations, var_to_idx)
        # print(terminal_node)
        
        # Step 4: Check for Linked Node
        linked_node = step4_find_linked_node(terminal_node, V_temp, global_partial_correlations, var_to_idx)
        
        if linked_node:  # If a linked node exists
            edges.append((linked_node, terminal_node))
            V_temp.remove(terminal_node)
        else:  # If no linked node exists
            new_latent = "L" + str(len(latent_nodes) + 1)
            latent_nodes.append(new_latent)
            edges.append((new_latent, terminal_node))
            V_temp.remove(terminal_node)
            
            K = compute_set_K(V_temp, terminal_node, global_partial_correlations, var_to_idx)
            if not K:
                return K,candidates, edges, latent_nodes

            else: 
                for node_j in K:
                    edges.append((node_j, new_latent))
                    idx_j = var_to_idx[node_j]
                    idx_k = var_to_idx[terminal_node]
                    candidates[node_j] = [node_j]
                    for node_i in V_temp:
                        if node_i != node_j and node_i != terminal_node and node_j != terminal_node:
                            idx_i = var_to_idx[node_i]
                            if global_partial_correlations[idx_k, idx_j, idx_i] == 0:
                                candidates[node_j].append(node_i)
            break  # Exit the loop after processing when no linked node exists
    return K, candidates, edges, latent_nodes


In [ ]:
def generate_final_latent_structures(sets, global_partial_correlations, var_to_idx):
    all_edges = {}
    all_latent_nodes = {}

    for key, value in sets.items():
        # Initial edges and latent nodes
        _, candidates, initial_edges, initial_latent_nodes = latent_tree_structure_prepare(
            value, global_partial_correlations
            , var_to_idx
            )
        
        # Rename the initial latent nodes
        renamed_initial_latent_nodes = [
            "L_" + key.split('_')[-1] + "_" + str(idx) 
            for idx, _ in enumerate(initial_latent_nodes, 1)]
        rename_dict_initial = dict(zip(initial_latent_nodes, renamed_initial_latent_nodes))
        renamed_initial_edges = [
            (rename_dict_initial[edge[0]] if edge[0] in rename_dict_initial else edge[0], 
                                  rename_dict_initial[edge[1]] if edge[1] in rename_dict_initial else edge[1]) 
                                  for edge in initial_edges]
        
        # Temporary storage for edges and latent nodes for each candidate set
        temp_edges = []
        temp_latent_nodes = []

        # If the candidate set is not empty, proceed to iterate over its values
        if candidates:
            for candidate_key, candidate_value in candidates.items():
                # Edges and latent nodes for the current candidate set
                _, _, candidate_edges, candidate_latent_nodes = latent_tree_structure_prepare(candidate_value, global_partial_correlations, var_to_idx)
                
                # Renaming the latent nodes based on the main set's index and the candidate set's index
                renamed_candidate_latent_nodes = [
                    "L_" + key.split('_')[-1] + "_" + candidate_key + "_" + str(idx) 
                    for idx, _ in enumerate(candidate_latent_nodes, 1)
                    ]
                rename_dict_candidate = dict(zip(candidate_latent_nodes, renamed_candidate_latent_nodes))
                renamed_candidate_edges = [
                    (rename_dict_candidate[edge[0]] if edge[0] in rename_dict_candidate else edge[0], 
                                           rename_dict_candidate[edge[1]] if edge[1] in rename_dict_candidate else edge[1]) 
                                           for edge in candidate_edges
                                           ]
                
                temp_edges.extend(renamed_candidate_edges)
                temp_latent_nodes.extend(renamed_candidate_latent_nodes)

        # Combine initial edges and latent nodes with those obtained from the candidate sets
        combined_edges = renamed_initial_edges + temp_edges
        combined_latent_nodes = renamed_initial_latent_nodes + temp_latent_nodes
        
        # Store the combined edges and latent nodes
        all_edges["E_" + key.split('_')[-1]] = combined_edges
        all_latent_nodes["latent_nodes_" + key.split('_')[-1]] = combined_latent_nodes

        """
        Remove duplicate edges from the edges dictionary.
        """
        # Using set to automatically remove duplicates and then converting back to list
        cleaned_edges = {key: list(set(value)) for key, value in all_edges.items()}

    return cleaned_edges, all_latent_nodes

In [ ]:
# Run the function
cleaned_final_edges, all_latent_nodes = generate_final_latent_structures(task1_sets, global_partial_correlations, var_to_idx)
# cleaned_final_edges, all_latent_nodes

- Combine nodes, edges and latent nodes for each subtree
- collapse the latent nodes to hidden cluster and clean the edges
- visulize the result for task 2 

In [ ]:
subtrees = {}  
for i in range(1, len(task1_sets) + 1):

    set_key = 'S_' + str(i)
    edge_key = 'E_' + str(i)
    latent_node_key = 'latent_nodes_' + str(i)
        
    subtree = {
        "nodes": task1_sets[set_key],
        "edges": cleaned_final_edges[edge_key],
        "latent_nodes": all_latent_nodes[latent_node_key]
    }
    subtrees['T_' + str(i)] = subtree

In [ ]:
collapsed_subtrees = {}
latent_cluster_counter = 1  # Counter for the latent cluster name

for key, subtree in subtrees.items():
    # Create a new name for the latent cluster based on the counter if there are latent nodes in the subtree
    latent_cluster_name = f"C_{latent_cluster_counter}" if subtree['latent_nodes'] else ""
    
    # Create a new list of nodes, removing the old latent nodes and adding the new cluster node
    new_nodes = [node for node in subtree['nodes'] if node not in subtree['latent_nodes']]
    if subtree['latent_nodes']:  # Only add the cluster node if there were latent nodes to begin with
        new_nodes.append(latent_cluster_name)
        latent_cluster_counter += 1  # Increment the counter for the next latent cluster
    
    # Update the edges, replacing any old latent nodes with the new cluster node
    new_edges = []
    for edge in subtree['edges']:
        new_edge = tuple([latent_cluster_name if node in subtree['latent_nodes'] else node for node in edge])
        new_edges.append(new_edge)
    
    collapsed_subtrees[key] = {
        'nodes': new_nodes,
        'edges': new_edges,
        'latent_clusters': [latent_cluster_name] if subtree['latent_nodes'] else []
    }

### Plot all subtrees in one big paper with proper distance

In [ ]:
def visualize_all_subtrees_with_axis_limits(ax, subtree, title, scale_factor=1.2):
    """
    Visualize the given subtree on a specific axis using networkx and matplotlib.
    Scale the layout to adjust edge lengths and set axis limits for clear separation.
    
    Parameters:
    - ax: Axis on which to plot the subtree.
    - subtree: Dictionary containing nodes, edges, and latent nodes information.
    - title: Title for the visualization.
    - scale_factor: Factor to scale the layout and adjust edge lengths.
    """
    G = nx.Graph()
    
    for node in subtree['nodes']:
        if node in subtree['latent_clusters']:
            G.add_node(node, color='#FFC0CB')  # Pink color for latent nodes
        else:
            G.add_node(node, color='#AEDFF7')  # Light blue color for regular nodes

    G.add_edges_from(subtree['edges'])
    
    # Use kamada_kawai_layout for better spacing between nodes
    layout = nx.kamada_kawai_layout(G)
    
    # Adjust the node positions by scaling the layout to adjust edge lengths
    for node, (x, y) in layout.items():
        layout[node] = (scale_factor * x, scale_factor * y)
    
    colors = [G.nodes[node]['color'] for node in G.nodes()]
    nx.draw(G, pos=layout, with_labels=True, node_color=colors, node_size=500, 
            font_size=10, font_weight='bold', edge_color='#A9A9A9', ax=ax, 
            width=1.3)  # Thicker edges for clarity
    ax.set_title(title, fontsize=12)  # Adjusted font size for title

    # Set the axis' background color
    ax.set_facecolor('#F5F5F5')  # Light gray background
    
    # Set axis limits to create a boundary around each graph
    ax.set_xlim([-2, 2]) # biggger the number, smaller the graph
    ax.set_ylim([-2, 2])

In [ ]:
# Initialize a large figure with 5x4 grid
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(18, 18))
fig.subplots_adjust(hspace=0.8, wspace=0.8)  # Increased spacing between subplots

# Visualize each subtree with adjusted edge lengths and axis limits
for ax, (key, subtree) in zip(axes.ravel(), collapsed_subtrees.items()):
    visualize_all_subtrees_with_axis_limits(ax, subtree, key)

# Hide any remaining unused subplots (since we're using a 5x4 grid for 16 subtrees)
for i in range(len(collapsed_subtrees), 5*4):
    axes.ravel()[i].axis('off')

# display
plt.tight_layout()
# plt.savefig("axis_limits_subtrees.png", dpi=300)
plt.show()

- Categorizing subtrees into with/without hidden nodes
- merge substree without hidden nodes first cause no merging crieria needed
- merge subtress with hidden nodes, follow the merging crieria 
- make sure in the end there is no cycles in our final quasi-skeleton of the latent polytree

In [ ]:
def visualize_subtree(subtree):
    """
    Visualize the given subtree using the networkx and matplotlib libraries.
    Nodes are colored based on their type (latent or visible), and edges are shown connecting the nodes.
    
    Parameters:
    - subtree: A dictionary containing nodes, edges, and latent_clusters of the subtree.
    """
    
    fig, ax = plt.subplots(figsize=(8, 6))
    G = nx.Graph()
    
    # Ensure all nodes present in edges are in the 'nodes' list
    all_nodes = set(subtree['nodes'])
    for edge in subtree['edges']:
        all_nodes.update(edge)
    
    # Add nodes to the graph
    for node in all_nodes:
        if node in subtree['latent_clusters']:
            G.add_node(node, color='#FFC0CB')  # Pink color for latent nodes
        else:
            G.add_node(node, color='#AEDFF7')  # Light blue color for regular nodes

    # Add edges to the graph
    G.add_edges_from(subtree['edges'])
    
    # Use kamada_kawai_layout for better spacing between nodes
    layout = nx.kamada_kawai_layout(G)
    
    colors = [G.nodes[node]['color'] for node in G.nodes()]
    nx.draw(G, pos=layout, with_labels=True, node_color=colors, node_size=500, 
            font_size=10, font_weight='bold', edge_color='#A9A9A9', ax=ax, 
            width=1.3)  # Thicker edges for clarity
    ax.set_title("Subtree", fontsize=12)  # Adjusted font size for title
    
    plt.show()

In [ ]:
def categorizing_subtrees(subtrees):
    subtree_with_hidden_clusters = {}
    subtree_without_hidden_clusters = {}

    for key, subtree in subtrees.items():
        if subtree['latent_clusters']:
            subtree_with_hidden_clusters[key] = subtree
            # Remove latent clusters from the nodes list
            for cluster in subtree['latent_clusters']:
                subtree['nodes'].remove(cluster)
        else:
            subtree_without_hidden_clusters[key] = subtree
            
    return subtree_with_hidden_clusters, subtree_without_hidden_clusters

In [ ]:
subtree_with_hidden_clusters, subtree_without_hidden_clusters = categorizing_subtrees(collapsed_subtrees)

In [ ]:
def merged_subtrees_without_hidden(subtrees_without_hidden):
    combined_nodes = []
    combined_edges = []

    # Iterate through each subtree and collect nodes and edges
    for _, subtree in subtrees_without_hidden.items():
        combined_nodes.extend(subtree['nodes'])
        combined_edges.extend(subtree['edges'])

    # Convert to set and back to list to ensure uniqueness 
    combined_nodes = list(set(combined_nodes))
    combined_edges = list(set(combined_edges))
    
    # Return the combined subtree
    return{
            "nodes": combined_nodes,
            "edges": combined_edges,
            "latent_clusters": []
    }

In [ ]:
def merging_criteria(subtree1, subtree2):
    # Extracting hidden clusters
    cluster1 = subtree1['latent_clusters'][0] if subtree1['latent_clusters'] else None 
    cluster2 = subtree2['latent_clusters'][0] if subtree2['latent_clusters'] else None 

    # If either of the subtrees doesn't have a hidden cluster, merging is not possible  
    if not cluster1 or not cluster2:
        return subtree1, subtree2, False
    
    # Find neighbors of hidden clusters in borh subtrees
    neighbors_cluster1 = {edge[1] for edge in subtree1['edges'] if edge[0] == cluster1} | \
                         {edge[0] for edge in subtree1['edges'] if edge[1] == cluster1}
    neighbors_cluster2 = {edge[1] for edge in subtree2['edges'] if edge[0] == cluster2} | \
                         {edge[0] for edge in subtree2['edges'] if edge[1] == cluster2} 

    
    # Check common neighbors
    common_neighbors = neighbors_cluster1.intersection(neighbors_cluster2)

    if len(common_neighbors) >= 2:
        # merged nodes 
        merged_nodes = list(set(subtree1['nodes'] + subtree2['nodes']))

        # Merge edges, renaming edges connected to cluster2 to clusters 1
        merged_edges = subtree1['edges'] + \
                [(cluster1 if node == cluster2 else node,
                 cluster1 if node2 == cluster2 else node2) 
                 for (node, node2) in subtree2['edges']]


        # create merged subtrees
        merged_subtree = {
            'nodes': merged_nodes,
            'edges': merged_edges,
            'latent_clusters': [cluster1]
        }

        return merged_subtree, {}, True  # Return merged subtree and a flag indicating merging happened
    
    return subtree1, subtree2, False  # If no merging, return original subtrees and a flag indicating no merging


In [ ]:
def merge_subtree_with_hidden_clusters_iterative(subtrees):
    """
    This function iteratively merges subtrees based on the merging_criteria. 
    For each subtree, it will attempt to merge with every other subtree until no more merges are possible.
    """
    merged_subtrees = {}
    keys = list(subtrees.keys())
    
    while keys:
        current_key = keys.pop(0)
        current_subtree = subtrees[current_key]
        
        merged = True  # Assume a merge is possible to enter the while loop
        while merged:
            merged = False  # Reset the merged flag for each iteration  
            # Check for a merge with every other subtree
            for other_key in keys:
                merged_subtree, _, did_merge = merging_criteria(current_subtree, subtrees[other_key])
                if did_merge:
                    print("current_clsuters: " + ', ' .join(current_subtree['latent_clusters']))
                    print("being merged cluster" + ', ' .join(subtrees[other_key]['latent_clusters']))
                    current_subtree = merged_subtree  # Update the current subtree to the merged one
                    keys.remove(other_key)  # Remove the merged subtree from keys
                    merged = True  # Mark that a merge has happened
                    break  # Exit the loop and check if the merged subtree can be merged further

        # After no more merges are possible, add the current subtree to the merged subtrees
        merged_subtrees[current_key] = current_subtree

    return merged_subtrees


In [ ]:
merged_subtrees_with_hidden_nodes = merge_subtree_with_hidden_clusters_iterative(subtree_with_hidden_clusters)
merged_subtrees_without_hidden_clusters = merged_subtrees_without_hidden(subtree_without_hidden_clusters)

In [ ]:
# Merge the subtrees above first, and then we can dectect cycles and start deleting edges
combined_tree = {
    "nodes": merged_subtrees_without_hidden_clusters["nodes"],
    "edges": merged_subtrees_without_hidden_clusters["edges"],
    "latent_clusters": merged_subtrees_without_hidden_clusters["latent_clusters"]
}

# Iterate over each key in merged_subtrees_with_hidden_nodes
for key in merged_subtrees_with_hidden_nodes:
    # Combine nodes
    combined_tree["nodes"].extend(merged_subtrees_with_hidden_nodes[key]["nodes"])
    # Ensure there are no duplicate nodes
    combined_tree["nodes"] = list(set(combined_tree["nodes"]))

    # Combine edges
    combined_tree["edges"].extend(merged_subtrees_with_hidden_nodes[key]["edges"])
    combined_tree["edges"] = list(set(combined_tree["edges"]))

    # Combine latent_clusters
    combined_tree["latent_clusters"].extend(merged_subtrees_with_hidden_nodes[key]["latent_clusters"])
    # Ensure there are no duplicate latent_clusters
    combined_tree["latent_clusters"] = list(set(combined_tree["latent_clusters"]))

In [ ]:
combined_tree["nodes"] += combined_tree["latent_clusters"]

In [ ]:
# Removing duplicate and reversed edges from combined_tree
unique_edges = set()
cleaned_edges = []

for edge in combined_tree['edges']:
    if edge not in unique_edges and (edge[1], edge[0]) not in unique_edges:
        cleaned_edges.append(edge)
        unique_edges.add(edge)

combined_tree['edges'] = cleaned_edges


In [ ]:
visualize_subtree(combined_tree)

- now delete cycles in a good way
- we clean the edges first, no duplicates
- detect the cycle, only compare the correlation among pairs of visible nodes 
- delete the one has the smallest absolute value of correlation

In [ ]:
def detect_cycle(graoh_with_cycles):

    # Extract nodes, edges, and latent_clusters from the combined_tree
    nodes = graoh_with_cycles['nodes']
    edges = graoh_with_cycles['edges']
    latent_clusters = graoh_with_cycles['latent_clusters']

    # Initialize a default dictionary to hold the adjacency list representation of the graph.
    graph = defaultdict(list)
    
    # Get the number of nodes in the graph.
    V = len(nodes)
    
    # Create a dictionary to map node names to indices for easier access.
    node_indices = {node: i for i, node in enumerate(nodes)}
    
    # Initialize a list to keep track of visited nodes during DFS.
    visited = [False] * V
    
    # Construct the graph from the given edges.
    # For an undirected graph, add the edge in both directions.
    for u, v in edges:
        graph[u].append(v)
        graph[v].append(u)
        
    def dfs(v, visited, parent, path):
        # Mark the current node as visited and add it to the current path.
        visited[node_indices[v]] = True
        path.append(v)
        
        for i in graph[v]:
            if not visited[node_indices[i]]:
                if dfs(i, visited, v, path):
                    return True
            elif i != parent:  # A cycle is detected.
                # Capture the cycle by tracing back from the current node to the first occurrence of 'i'.
                cycle_start_idx = path.index(i)
                path.append(i)
                del path[:cycle_start_idx]
                return True
                
        # Backtrack
        path.pop()
        return False
    
    for node in nodes:
        if not visited[node_indices[node]]:
            path = []
            if dfs(node, visited, None, path):
                return True, path
                
    return False, []


In [ ]:
def get_index(node_name, latent_clusters):
    # Extract index from node name; modify this based on your actual node naming convention.
    return int(node_name[2:]) if node_name in latent_clusters else -1

In [ ]:
def edge_to_remove_from_cycle(cycle, latent_clusters,correlation_matrix):
    cycle_nodes = cycle[1]

     # Filter out edges that involve nodes in the latent_clusters.
    valid_edges = [(cycle_nodes[i], cycle_nodes[i + 1]) for i in range(len(cycle_nodes) - 1) 
                    if cycle_nodes[i] not in latent_clusters and cycle_nodes[i + 1] not in latent_clusters]

    # if there are valid edges
    if valid_edges:
        correlations = [abs(correlation_matrix.loc[edge[0], edge[1]]) for edge in valid_edges]
        # Identify the edge with the smallest absolute correlation.
        min_index = correlations.index(min(correlations))
        return valid_edges[min_index]

    
    else:
        hidden_edges = [(cycle_nodes[i], cycle_nodes[i + 1]) for i in range(len(cycle_nodes) - 1) 
                    if cycle_nodes[i] in latent_clusters or cycle_nodes[i + 1] in latent_clusters]
        
        # Assuming hidden node names are in a format where the index can be extracted and compared.
        max_hidden_node_edge = max(
            hidden_edges, 
            key=lambda edge: max(get_index(edge[0], latent_clusters), 
            get_index(edge[1], latent_clusters))
            )
        return max_hidden_node_edge


In [ ]:
def cycle_removal(graph, original_correlation_matrix):
    while True:
        cycle = detect_cycle(graph)
        print(cycle)
        
        if not cycle[0]:# If no cycle detected
            break
        edge_to_delete = edge_to_remove_from_cycle(cycle, graph['latent_clusters'], original_correlation_matrix)
        
        if edge_to_delete in graph['edges']:
            graph['edges'].remove(edge_to_delete)
        elif (edge_to_delete[1], edge_to_delete[0]) in graph['edges']:
            graph['edges'].remove((edge_to_delete[1], edge_to_delete[0]))  
    return graph

In [ ]:
updated_tree = cycle_removal(combined_tree, original_correlation_matrix)

In [ ]:
visualize_subtree(updated_tree)

In [ ]:
task3_output = updated_tree

- update the output of task 2 so it has updated clusters
- because task4 initially only consider the clusters after the merge of the cluster

In [ ]:
def remove_latent_cluster(subtrees_dic_with_latent_clusters):
    """
    Removes latent_clusters and edges associated with them from the subtrees.
    
    Args:
    - collapsed_subtrees (dict): A dictionary representing the subtrees.

    Returns:
    - dict: A new dictionary with latent_clusters removed and corresponding edges filtered out.
    """

    # create a new dictionary to store the modified data
    input_task4_from_task2 = {}

    # iterate through each subtree in the dictionary
    for key, value in subtrees_dic_with_latent_clusters.items():
        # create a deep copy of the current subtree to ensure we don't modify the original data
        new_value = value.copy()
        
        # empty the latent_clusters list
        new_value['latent_clusters'] = []
        
        # filter out edges that contain any latent_cluster
        new_value['edges'] = [
            edge for edge in value['edges'] 
            if not any(node in value['latent_clusters'] for node in edge)
        ]
        
        # add the modified subtree to the new dictionary
        input_task4_from_task2[key] = new_value

    return input_task4_from_task2


In [ ]:
def hidden_root_detection (V_tilda, r):

    """
    Function to perform hidden root detection based on V_tilda and a current key.

    :param V_tilda: Dictionary containing V_tilda_i as keys and their corresponding sets.
    :param current_key: Current subtree iteration, e.g., 'T_1', 'T_2', etc.
    
    :return: Returns two dictionaries, left_relative_complement and right_relative_complement.
    """
    # Initialize the result dictionaries 
    left_relative_complement  = {}
    right_relative_complement  = {}

    # Calculate the relative complements 
    V_tilda_r = V_tilda[f'V_tilda_{r}']
    for key, value in V_tilda.items():
        i = int(key.split('_')[2])
        if i != r:
            left_difference = V_tilda_r - value
            right_difference = value - V_tilda_r
        
            left_length = 0 if not left_difference else len(left_difference)
            right_length = 0 if not right_difference else len(right_difference)
        
            left_relative_complement[f'relative_complement_{r}_{i}'] = left_length
            right_relative_complement[f'relative_complement_{i}_{r}'] = right_length

    # check conditions
    all_conditions_met = True
    
    for key, value in left_relative_complement.items():
        # print(key)
        i = int(key.split('_')[3])
        # print(i)

        if i != r:
            left_key =  f'relative_complement_{r}_{i}'
            right_key = f'relative_complement_{i}_{r}'

            left_value = left_relative_complement[left_key]
            # print(left_value)
            right_value = right_relative_complement[right_key]
            # print(right_value)
        
            # check 
            if (left_value <= 1 and right_value >1):
                # print(f"Condition not met for: {key}, left value: {value}, right value: {right_value}")  # Print the issue
                return False
            
    if all_conditions_met:
            return True, r, left_relative_complement, right_relative_complement

In [ ]:
def lemma_18(r, left_relative_complement, right_relative_complement, V_tilda, subtree):
    
    big_I = {r}
    big_I_copy = {r}

    for key in left_relative_complement:
        i = int(key.split('_')[3])

        if i!= r:
            left_key =  f'relative_complement_{r}_{i}'
            right_key = f'relative_complement_{i}_{r}'

            left_value = left_relative_complement[left_key]
            # print(left_value)
            right_value = right_relative_complement[right_key]
            # print(right_value)

            if left_value == 1 and right_value == 1:
                big_I.add(i)

    for key in left_relative_complement:
        i = int(key.split('_')[3])

        if i!= r:
            left_key =  f'relative_complement_{r}_{i}'
            right_key = f'relative_complement_{i}_{r}'

            left_value = left_relative_complement[left_key]
            # print(left_value)
            right_value = right_relative_complement[right_key]
            # print(right_value)

            if left_value ==1  and right_value in (0,1):
                big_I_copy.add(i)
            

    # Compute big_W and big_Wbar
    big_W = set().union(*[V_tilda[f'V_tilda_{i}'] for i in big_I])
    big_Wbar = set().union(*[V_tilda[i] for i in V_tilda.keys() if int(i.split('_')[2]) not in big_I])
    intersection_W = big_W & big_Wbar

    # Calculating nodes_linked_to_hidden
    nodes_linked_to_hidden = big_W - big_Wbar

    # Need a second choice of nodes_linked_to_hidden
    big_W_copy = set().union(*[V_tilda[f'V_tilda_{i}'] for i in big_I_copy])
    big_Wbar_copy = set().union(*[V_tilda[i] for i in V_tilda.keys() if int(i.split('_')[2]) not in big_I_copy])
    intersection_W_copy = big_W_copy & big_Wbar_copy


    if not nodes_linked_to_hidden: 
        # print("loose the condition")
        differerence = big_W_copy - big_Wbar_copy
        if differerence:
            nodes_linked_to_hidden = {list(differerence)[0]}
            print(nodes_linked_to_hidden)
            return nodes_linked_to_hidden, intersection_W_copy,big_I_copy
        else:
            # print("loose the condition again")
            return False
        
    return nodes_linked_to_hidden, intersection_W, big_I

In [ ]:
def task_4b(intersection_W, big_I, subtrees, inter_count):
    # print('intersection_W:')
    # print(intersection_W)
    # reorder and deduplicate intersection_W
    intersection_W = sorted(set(intersection_W), key=lambda x: int(x[1:]))

    # initialze n_k
    n_k = len(intersection_W)

    # create n_k number of hidden clusters and add edges 
    for i in big_I:
        for k in range(n_k):
            new_cluster =  f"C_{inter_count}_{i}_{k+1}"
            new_edge =  (new_cluster, intersection_W[k])
            subtree_key = f"T_{i}"
            subtrees[subtree_key]['latent_clusters'].append(new_cluster)
            subtrees[subtree_key]['edges'].append(new_edge)
    return subtrees

In [ ]:
def new_hidden_cluster_merging_criteria(cluster1, cluster2, V_tilda_to_compare,subtree):
    # Determine the visible neighbors of each cluster within the subtree
    visible_neighbors_cluster1 = [
        target for node, target in subtree['edges'] 
        if node == cluster1 and target in subtree['nodes']
        ]
    visible_neighbors_cluster2 = [
        target for node, target in subtree['edges'] 
        if node == cluster2 and target in subtree['nodes']
        ]

    # compute the union of the visible neighbors
    visible_neighbors = set (visible_neighbors_cluster1 + visible_neighbors_cluster2)

    # check the overlap with the nodes iin V_tilda_to_compare 
    distinct_nodes_count = sum(1 for V_tilda_nodes in V_tilda_to_compare.values()if len(V_tilda_nodes.intersection(visible_neighbors)) > 0)

    # return True if there are at least 2 distinct nodes, otherwise return False 
    return distinct_nodes_count >= 2

In [ ]:
def task_4c(subtree_from_4b, big_I, V_tilda):
    V_tilda_to_compare = {k: v for k, v in V_tilda.items() if int(k.split('_')[-1]) not in big_I}
    
    # Update subtrees based on big_I
    for i in big_I:
        subtree_key = f"T_{i}"
        subtree = subtree_from_4b[subtree_key] 
        latent_clusters = subtree['latent_clusters']

        while latent_clusters:
            any_merge_happened =  False # This flag checks if any merge happended in the entire iteration
            current_cluster = latent_clusters[0]
            merged = True # assume a merger is possible to enter the while loop 

            while merged:
                merged = False # reset the mergerd flag for each iteration
                for other_cluster in [oc for oc in latent_clusters if oc != current_cluster]:
                    if new_hidden_cluster_merging_criteria(current_cluster, other_cluster, V_tilda_to_compare, subtree):
                        latent_clusters.remove(other_cluster) # remove the merged cluster from latent_clusters
                        # modify the edges by using the smaller index's latent cluster 
                        subtree['edges'] = [
                            (current_cluster if node == other_cluster else node, target) 
                            for node, target in subtree['edges']
                            ]
                        merged = True # mark that a merge has happened 
                        any_merge_happened = True
                        break # exit the loop and check if the merged cluster can be merged clusters 
            if not any_merge_happened:
                break
                        
            # after no more merges are possible, add the current cluster to the merged clusters    
        subtree['latent_clusters'] =  latent_clusters
        subtree_from_4b[subtree_key] = subtree # Store the updated subtree in the resulting dictionary

    return subtree_from_4b  

In [ ]:
def HMCA(current_tree_key, subtrees, neighbors_of_current_clusters, current_hidden_cluster,inter_count):
    current_key = current_tree_key
    r = int(current_key.split('_')[1])

    # Initialize V_i for each subtree T_i
    V = {key: set(subtree['nodes']) for key, subtree in subtrees.items()}

    # Compute V~i for each subtree T_i
    V_tilda = {f'V_tilda_{key.split("_")[1]}': nodes.intersection(neighbors_of_current_clusters) 
           for key, nodes in V.items()}  

    independence_relation_key = f'independence_{r}'
    independence_relation = set()

    if not V_tilda.get(f'V_tilda_{r}'):   # Check if V_tilda_r is empty
        return subtrees, {independence_relation_key: independence_relation}
    
    if all (not v for v in V_tilda.values()):  # Step 2, Check if all V~i are empty 
        return subtrees, {independence_relation_key: independence_relation}
    
    # only need those that has intersection not empty
    V_tilda = {key: value for key, value in V_tilda.items() if value}

    # Call hidden_root  
    result = hidden_root_detection(V_tilda, r)

    if result:
        _, r, left_relative_complement, right_relative_complement = result
        lemma_result= lemma_18(r, left_relative_complement, right_relative_complement, V_tilda, subtrees[current_key])

        if not lemma_result:
            # print("lemma 18 returned False. Need to debug")
            return subtrees,{independence_relation_key: independence_relation}

        nodes_linked_to_hidden, intersection_W, big_I = lemma_result

        if nodes_linked_to_hidden:
            new_node = f'y_h_{inter_count}_{r}' # create the new node 
            # create edges between the new node and each node in nodes_linked_to_hidden
            new_edges = [(new_node, node) for node in nodes_linked_to_hidden] 

            # add this node and edge to all T_i where in big_I
            for i in big_I:
                key = f'T_{i}'
                subtrees[key]['nodes'].append(new_node)
                subtrees[key]['edges'].extend(new_edges)

            # Create independence relations
            for y in subtrees[key]['nodes']:
                if y != new_node:
                    independence_relation.add((new_node, y))    
                    
            if len(intersection_W) > 0:
                subtrees_4b = task_4b(intersection_W, big_I, subtrees,inter_count)
                subtrees_4c = task_4c(subtrees_4b, big_I, V_tilda)
                return subtrees_4c, {independence_relation_key: independence_relation}
            else:
                return subtrees,{independence_relation_key: independence_relation}
        else:
            print("linked visible nodes can not be found, need to debug where went wrong")
            return subtrees,{independence_relation_key: independence_relation}
    else:
        return subtrees,{independence_relation_key: independence_relation}

In [ ]:
# a function do the categorize during task 4, we refer the one task3, but not exactly the same
def categorizing_subtrees_task4(subtrees):
    subtree_with_hidden_clusters = {}
    subtree_without_hidden_clusters = {}

    for key, subtree in subtrees.items():
        if subtree['latent_clusters']:
            subtree_with_hidden_clusters[key] = subtree
            # Remove latent clusters from the nodes list
            for cluster in subtree['latent_clusters']:
                if cluster in subtree['nodes']:
                    subtree['nodes'].remove(cluster)
        else:
            subtree_without_hidden_clusters[key] = subtree
            
    return subtree_with_hidden_clusters, subtree_without_hidden_clusters

In [ ]:
def task_3_for_task4(subtrees):

    subtree_with_hidden_clusters, subtree_without_hidden_clusters = categorizing_subtrees_task4(subtrees)
    merged_subtrees_with_hidden_nodes = merge_subtree_with_hidden_clusters_iterative(subtree_with_hidden_clusters)
    merged_subtrees_without_hidden_clusters = merged_subtrees_without_hidden(subtree_without_hidden_clusters)
    # Merge the subtrees above first, and then we can dectect cycles and start deleting edges
    combined_tree = {
        "nodes": merged_subtrees_without_hidden_clusters["nodes"],
        "edges": merged_subtrees_without_hidden_clusters["edges"],
        "latent_clusters": merged_subtrees_without_hidden_clusters["latent_clusters"]
    }

    # Iterate over each key in merged_subtrees_with_hidden_nodes
    for key in merged_subtrees_with_hidden_nodes:
        # Combine nodes
        combined_tree["nodes"].extend(merged_subtrees_with_hidden_nodes[key]["nodes"])
        # Ensure there are no duplicate nodes
        combined_tree["nodes"] = list(set(combined_tree["nodes"]))

        # Combine edges
        combined_tree["edges"].extend(merged_subtrees_with_hidden_nodes[key]["edges"])
        combined_tree["edges"] = list(set(combined_tree["edges"]))

        # Combine latent_clusters
        combined_tree["latent_clusters"].extend(merged_subtrees_with_hidden_nodes[key]["latent_clusters"])
        # Ensure there are no duplicate latent_clusters
        combined_tree["latent_clusters"] = list(set(combined_tree["latent_clusters"]))

        unique_edges = set()
        cleaned_edges = []

        for edge in combined_tree['edges']:
            if edge not in unique_edges and (edge[1], edge[0]) not in unique_edges:
                cleaned_edges.append(edge)
                unique_edges.add(edge)

        combined_tree['edges'] = cleaned_edges

    return combined_tree

In [ ]:
def find_neighbors_of_a_latent_clusters(quasi_skeleton, current_hidden_cluster):

    neighbors_of_current_clusters = set()
    for edge in quasi_skeleton['edges']:
        if current_hidden_cluster in edge: 
            neighbors_of_current_clusters.add(edge[0] if edge[1] == current_hidden_cluster else edge[1])
    
    return neighbors_of_current_clusters

In [ ]:
def task4(task2_subtree, task3_output):
    
    subtrees = task2_subtree.copy()
    quasi_skeleton = task3_output.copy()
    # create a dictionary to store indpendence_relations for each interartion 
    independence_relations_dict = {}

    # Step 1: initialize P
    P = set(quasi_skeleton['latent_clusters'])
    inter_count = 1 
    max_inter = 20

    # Step 2: Loop until P is empty
    while P:

        print(inter_count)
        print(P)
        
        # create a new dictionary for the current inter_count
        independence_relations_dict[inter_count] = {}

        current_hidden_cluster = list(P)[0] # taking the first element of P
        subtrees = remove_latent_cluster(subtrees)
        neighbors_of_current_clusters = find_neighbors_of_a_latent_clusters(quasi_skeleton,current_hidden_cluster)

        # For each subtree in subtree 
        for T_i in list(subtrees.keys()): # iterating over a copy of the keys to avoid dictionarsize change during iteration 
            # run HMCA on the subtree 

            updated_subtrees, new_independence_relations = HMCA(
                T_i, 
                subtrees, 
                neighbors_of_current_clusters, 
                current_hidden_cluster, 
                inter_count)
            
            # update subtrees with the output from HMCA
            subtrees = updated_subtrees

            # accumulate the new independence relations 
            independence_relations_dict[inter_count][f"independence_relation_{T_i}"] = new_independence_relations

        quasi_skeleton = task_3_for_task4(subtrees)
        print(quasi_skeleton['latent_clusters'])
        P = set(quasi_skeleton['latent_clusters'])
        # now we should update t

        # keep checking the count 
        inter_count  +=1
        if inter_count >= max_inter:
            break


    # Step 3: Return the updated task2_subtree
    return subtrees, independence_relations_dict, quasi_skeleton

In [ ]:
task4_subtrees, task4_independence_relations, task4_skeleton  = task4(collapsed_subtrees,task3_output)

In [ ]:
task4_independence_relations

In [ ]:
def detect_cycle(graph_with_cycles):
    # Extract nodes and edges from the graph
    nodes = graph_with_cycles['nodes']
    edges = graph_with_cycles['edges']

    # Construct a graph using networkx
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    
    # Use networkx's simple_cycles function to find cycles
    # It returns a list of cycles, where each cycle is a list of nodes
    cycles = list(nx.simple_cycles(G))
    
    if cycles:  # If there's at least one cycle
        return True, cycles[0]  # Return the first detected cycle
    else:
        return False, []

In [ ]:
task4_skeleton_cleaned = cycle_removal(task4_skeleton, original_correlation_matrix)

In [ ]:
visualize_subtree(task4_skeleton_cleaned)

# Task 5: Obtain the pattern of the latent polytree from task 4

## expand the correlatin matrix from task 1
- make a copy first
- if the pair occurs in the indepence relation we gain from task 4, put 1 in the entry, 0 otherwise 
- make sure all entries get populated by usinmg symmetry of correlation matrix

In [ ]:
# create a copy of the original matrix
expanded_matrix = new_correlation_matrix.copy()

# Extract hidden nodes from task4_independence_relations
hidden_nodes = set()
for iteration_relations in task4_independence_relations.values():
    for relations in iteration_relations.values():
        for relation in relations.values():
            for pair in relation:
                for node in pair:
                    if node in pair:
                        if node.startswith('y_h') and node not in hidden_nodes:
                            hidden_nodes.add(node)
                            
# Expand expanded_matrix based on the hidden nodes
for hidden_node in hidden_nodes:
    expanded_matrix[hidden_node] = 0 #add a new column 
    expanded_matrix.loc[hidden_node] = 0 #add a new row 


# Fill up the entries of the expanded correlation matrix based on task4_independence_relations
for iteration_relations in task4_independence_relations.values():
    for relations in iteration_relations.values():
        for relation in relations.values():
            for pair in relation:
                expanded_matrix.loc[pair[0], pair[1]] = 1
                expanded_matrix.loc[pair[1], pair[0]] = 1

# Ensure the diagonal values are 1
np.fill_diagonal(expanded_matrix.values, 1)

- rename the hidden nodes by keep counting forward for better indexing
- update task4_skeleton_cleaned['nodes'] 
- update task4_skeleton_cleaned['edges'] 

In [ ]:
def rename_hidden_nodes(nodes):
    # Find the largest visible node number
    max_visible_node = max(int(node[1:]) for node in nodes if not node.startswith('y_h'))
    
    # Create a mapping for renaming and a set for renamed hidden nodes
    rename_map = {}
    renamed_hidden_nodes = set()
    hidden_index = 1
    for node in nodes:
        if node.startswith('y_h'):
            new_name = f'y{max_visible_node + hidden_index}'
            rename_map[node] = new_name
            renamed_hidden_nodes.add(new_name)
            hidden_index += 1
            
    # Rename nodes
    renamed_nodes = [rename_map.get(node, node) for node in nodes]
    return renamed_nodes, rename_map, renamed_hidden_nodes

In [ ]:
# get input for rename_hidden_nodes function 
nodes = task4_skeleton_cleaned['nodes']

# retrive renamed_nodes/map/edges from the function above
renamed_nodes, rename_map, renamed_hidden_nodes = rename_hidden_nodes(nodes)

# Update edges with the renamed nodes
renamed_edges = [(rename_map.get(edge[0], edge[0]), rename_map.get(edge[1], edge[1])) for edge in task4_skeleton_cleaned['edges']]

In [ ]:
# Use the renamed map to update the column name of our expanded_matrix
expanded_matrix = expanded_matrix.rename(columns=rename_map, index=rename_map)

- have renamed_nodes as V
- have renamed_edges as E 
- use expanded_matrix to indicate independence relations

In [ ]:
def clean_edges(edges):
    cleaned_edges = set()
    
    for edge in edges:
        # Sort the edge tuple to maintain consistency
        cleaned_edge = tuple(sorted(edge))
        cleaned_edges.add(cleaned_edge)
    
    return list(cleaned_edges)

In [ ]:
renamed_edges_clean = clean_edges(renamed_edges)

In [ ]:
def orient_edges_debug(nodes, edges, correlation_matrix):
    is_additional_edges_are_oriented = True
    E_hat = set()
    
    while is_additional_edges_are_oriented:
        is_additional_edges_are_oriented = False
        action_happened = False
        
        for edge1 in edges:
            for edge2 in edges:
                if edge1 != edge2:
                    common_node = set(edge1).intersection(set(edge2))
                    if len(common_node) == 1:
                        common_node = common_node.pop()
                        node1, node2 = set(edge1).difference({common_node}), set(edge2).difference({common_node})
                        node1, node2 = node1.pop(), node2.pop()

                        try:
                            if correlation_matrix.loc[node1, node2] == 0:
                                if (node1, common_node) not in E_hat and (common_node, node1) not in E_hat:
                                    E_hat.add((node1, common_node))
                                    action_happened = True
                                if (node2, common_node) not in E_hat and (common_node, node2) not in E_hat:
                                    E_hat.add((node2, common_node))
                                    action_happened = True
                            elif (node1, common_node) in E_hat or (common_node, node1) in E_hat:
                                if (common_node, node2) not in E_hat and (node2, common_node) not in E_hat:
                                    E_hat.add((common_node, node2))
                                    action_happened = True
                        except KeyError as e:
                            print(f"KeyError occurred: {e}")
                            # Handle the error or continue
        
        if action_happened:
            is_additional_edges_are_oriented = True
    
    return E_hat


In [ ]:
from itertools import combinations


In [ ]:
def orient_edges(edges, correlation_matrix):
    
    is_additional_edges_are_oriented = True
    E_hat = set()
    
    while is_additional_edges_are_oriented:
        is_additional_edges_are_oriented = False
        action_happened = False

    for edge1, edge2 in combinations(edges, 2):
        common_node = set(edge1).intersection(set(edge2))

        if len(common_node) == 1:
            common_node = common_node.pop()
            node1, node2 = set(edge1).difference({common_node}), set(edge2).difference({common_node})
            node1, node2 = node1.pop(), node2.pop()
            
            if correlation_matrix.loc[node1, node2] == 0 or correlation_matrix.loc[node2, node1] == 0:
                if (node1, common_node) not in E_hat:
                    E_hat.add((node1, common_node))
                    action_happened = True
                if (node2, common_node) not in E_hat:
                    E_hat.add((node2, common_node))
                    action_happened = True

            elif (node1, common_node) in E_hat or (common_node, node2) in edges:
                if (common_node, node2) not in E_hat:
                    E_hat.add((common_node, node2))
                    action_happened = True
        
        if action_happened:
            is_additional_edges_are_oriented = True
    
    return E_hat

In [ ]:
E_hat = orient_edges(renamed_edges_clean, expanded_matrix)

In [ ]:
def check_bidirectional_edges(E_hat):
    bidirectional_edges = set()
    for edge in E_hat:
        if (edge[1], edge[0]) in E_hat:
            bidirectional_edges.add(edge)
            bidirectional_edges.add((edge[1], edge[0]))
    if bidirectional_edges:
        return bidirectional_edges
    else:
        return False

In [ ]:
check_bidirectional_edges_result = check_bidirectional_edges(E_hat)
check_bidirectional_edges_result

In [ ]:
def type_2_detection(E_hat, nodes_list, check_bidirectional_edges_result):
    
    nodes = set(nodes_list)  # Convert the list to a set
    max_node = max(nodes, key=lambda x: int(x[1:]))
    max_index = int(max_node[1:])

    if check_bidirectional_edges_result:
        
        for edge in check_bidirectional_edges_result:
            E_hat.remove(edge)

        unique_bidirectional_edges = set()

        for edge in check_bidirectional_edges_result:
            if (edge[1], edge[0]) not in unique_bidirectional_edges:
                unique_bidirectional_edges.add(edge)

        for edge in unique_bidirectional_edges:
            max_index += 1
            new_node = 'y' + str(max_index)
            nodes.add(new_node)
            E_hat.add((new_node, edge[0]))
            E_hat.add((new_node, edge[1]))
            renamed_hidden_nodes.add(new_node)
    
    else:
        return list(nodes), E_hat

    return list(nodes), E_hat  # Convert the set back to a list if needed


In [ ]:
nodes_final, E_hat_final = type_2_detection(E_hat, renamed_nodes, check_bidirectional_edges_result)

In [ ]:
def plot_directed_graph(nodes, directed_edges, renamed_hidden_nodes):
    # Create a directed graph
    G = nx.DiGraph()
    
    # Add nodes
    for node in nodes:
        G.add_node(node)
    
    # Add directed edges
    for edge in directed_edges:
        G.add_edge(*edge)
    
    # Create a list of node colors based on whether they are in renamed_hidden_nodes
    node_colors = ['lightpink' if node in renamed_hidden_nodes else 'skyblue' for node in G.nodes()]

    # Draw the graph
    pos = nx.nx_agraph.graphviz_layout(G, prog='dot')
    nx.draw(G, pos, with_labels=True, arrows=True, node_size=700, node_color=node_colors, font_size=10, font_color='black', font_weight='bold', arrowsize=20, connectionstyle='arc3,rad=0.1')
    plt.show()

    return G


In [ ]:
G_1 = plot_directed_graph(nodes_final, E_hat_final, renamed_hidden_nodes)

In [ ]:
df_edge = pd.read_csv(original_edge_data_path)
# df_edge = pandas2ri.rpy2py(original_edge_data)
df_edge = pd.DataFrame(df_edge)
# df_edge.columns = ['source', 'target']
df_edge['source'] = df_edge['source'].astype(str)
df_edge['target'] = df_edge['target'].astype(str)

G_original = nx.from_pandas_edgelist(df_edge, 'source', 'target', create_using=nx.DiGraph())

In [ ]:
number_hidden_recovered = len(renamed_hidden_nodes)

number_nodes_total = df_edge.shape[0] + 1
number_visible_nodes = len(df_data.columns)

true_number_hidden = number_nodes_total - number_visible_nodes
recover_ratio_hidden = number_hidden_recovered/true_number_hidden

true_list_hidde_nodes = [str(number_nodes_total - i) for i in range(true_number_hidden)]

In [ ]:
def calculate_metrics(G_estimated, G_true):
    """
    Calculate Structural Hamming Distance (SHD), Precision, and Recall between two graphs.
    SHD is the number of edge disagreements between the two graphs.
    Precision is the proportion of correctly identified edges out of all predicted edges.
    Recall is the proportion of actual edges that were correctly identified.
    """
    # Convert estimated graph edges to a format compatible with the true graph
    edges_estimated = {(u[1:], v[1:]) if u.startswith('y') and v.startswith('y') else (u, v)
                       for u, v in G_estimated.edges()}
    print(edges_estimated)
    
    # No conversion needed for true graph edges if they are already in the correct format
    edges_true = set(G_true.edges())
    print(edges_true)

    # Calculate SHD
    shd = len(edges_estimated.symmetric_difference(edges_true))
    
    # Calculate components for Precision and Recall
    true_positives = len(edges_estimated.intersection(edges_true))
    false_positives = len(edges_estimated) - true_positives
    false_negatives = len(edges_true) - true_positives

    # Precision calculation
    if true_positives + false_positives > 0:
        precision = true_positives / (true_positives + false_positives)
    else:
        precision = 0  # To avoid division by zero when there are no positive predictions

    # Recall calculation
    if true_positives + false_negatives > 0:
        recall = true_positives / (true_positives + false_negatives)
    else:
        recall = 0  # To avoid division by zero when there are no true positives or false negatives
    
    return shd, precision, recall


In [ ]:
# rename the graph object into true and estimate to be more clear
g_estimated = G_1.copy()
g_true = G_original.copy()

In [ ]:
def update_graph_with_hidden_nodes(g_graph, hidden_nodes):
    # Make a copy of the graph
    g_copy = g_graph.copy()

    # Helper function to add bidirectional edges between a list of nodes and remove the original node
    def add_bidirectional_edges_and_remove_node(node, children):
        for node1, node2 in combinations(children, 2):
            g_copy.add_edge(node1, node2)
            g_copy.add_edge(node2, node1)
        # Remove the original node and its edges to/from children
        for child in children:
            if g_copy.has_edge(node, child):
                g_copy.remove_edge(node, child)
            if g_copy.has_edge(child, node):  # If the graph allows for backward edges
                g_copy.remove_edge(child, node)
        g_copy.remove_node(node)

    def process_node(node):
        parents = list(g_copy.predecessors(node))
        children = list(g_copy.successors(node))
        
        # Remove the node and its edges if it has one or no children
        if not parents and len(children) <= 1:
            g_copy.remove_node(node)
            for child in children:
                if g_copy.has_edge(node, child):
                    g_copy.remove_edge(node, child)
            return

        # For nodes with multiple children but no parents, add bidirectional edges between children, then remove the node and its edges
        if not parents and len(children) > 1:
            add_bidirectional_edges_and_remove_node(node, children)
            return
        
        # For nodes with parents, add edges from each parent to each child, then remove the node and its related edges
        if parents:
            for parent in parents:
                for child in children:
                    g_copy.add_edge(parent, child)
            # After adding edges from parents to children, remove the hidden node and its edges
            g_copy.remove_node(node)
            for parent in parents:
                if g_copy.has_edge(parent, node):
                    g_copy.remove_edge(parent, node)
                if g_copy.has_edge(node, parent):  # If the graph allows for backward edges
                    g_copy.remove_edge(node, parent)
            for child in children:
                if g_copy.has_edge(node, child):
                    g_copy.remove_edge(node, child)
                if g_copy.has_edge(child, node):  # If the graph allows for backward edges
                    g_copy.remove_edge(child, node)
    
    for hidden_node in hidden_nodes:
        if hidden_node in g_copy.nodes():
            process_node(hidden_node)

    g_copy = nx.DiGraph(g_copy)
    
    return g_copy

In [ ]:
updated_graph_true = update_graph_with_hidden_nodes(g_true, true_list_hidde_nodes)
updated_graph_estimated = update_graph_with_hidden_nodes(g_estimated, renamed_hidden_nodes)

In [ ]:
num_edges_esitimation = updated_graph_estimated.number_of_edges()
num_edges_true = updated_graph_true.number_of_edges()

In [ ]:
import json

In [ ]:
shd_value, precision, recall  = calculate_metrics(updated_graph_estimated, updated_graph_true)

In [ ]:
# Assuming shd_value and recover_ratio_hidden are defined earlier in your notebook
algorithm_results = {
    "shd": shd_value,  
    "precision": precision,
    "recall": recall,
    "percentage_hidden_nodes_recovered": recover_ratio_hidden,
    "number_of_edges_esitimation":num_edges_esitimation,
    "number_of_edges_true":num_edges_true
}

In [ ]:
# Correctly outputting JSON string
print("algorithm_result", json.dumps(algorithm_results))